In [1]:
import tensorflow as tf
import numpy as np
import mlp.tf_utils as utils
from mlp.data_providers import AugmentedCIFAR10DataProvider, AugmentedCIFAR100DataProvider, CIFAR100DataProvider, CIFAR10DataProvider
from mlp.image_transforms import random_flip, random_crop, center_crop, random_flip_small
from mlp.Conv_models import ConvModel, TwoTaskConvModel, TwoTaskConvModelSoftSharing
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

seed = 12345 
rng = np.random.RandomState(seed)

In [2]:
layer_dims = [3, 64, 128, 128, 128, _]
batch_size = 256

bifurcation_point = 3
joint_loss = False
max_pools = [True, True, False, True]
lrns = []
lrn_alphas = []
batch_norms = [True, True, True, True]
l1 = False
l2 = 5e-4
learning_rates = [1e-1, 1e-2, 1e-3]
lrn_schedule = [-1, 66, 133]
optimizer_params = [0.9, 0.999]  
activation = tf.nn.relu
dropout = True
epochs = 200
error = 'soft_max_cross_entropy'
image_size = 32  
optimiser = "ADAM"  
logs_dir = "tf-log"
soft_loss = 1e-1

In [3]:
for i, ratio in enumerate([1.]):
    
    ########### MTL CLASSIFIER CIFAR100 & CIFAR10 #################
    tf.reset_default_graph()
    model_name = "MTL_CIFAR_100_and_10_ADAM"
    print("MODEL:- " + model_name)
    
    main_train_data = CIFAR100DataProvider(which_set='train', batch_size=batch_size, shuffle_order=False)
    main_valid_data = CIFAR100DataProvider(which_set='valid', batch_size=batch_size, shuffle_order=False)

    aux_train_data = CIFAR10DataProvider(which_set='train', batch_size=batch_size, shuffle_order=False)
    aux_valid_data = CIFAR10DataProvider(which_set='valid', batch_size=batch_size, shuffle_order=False)
    
    m_t_idx = utils.sample_data(main_train_data, ratio)
    m_v_idx = utils.sample_data(main_valid_data, ratio)
    _1 = utils.sample_data(aux_train_data, ratio, m_t_idx)
    _2 = utils.sample_data(aux_valid_data, ratio, m_v_idx)
    
    model = TwoTaskConvModelSoftSharing(conv_layer_dims=layer_dims, main_train_data=main_train_data, main_valid_data=main_valid_data,
                         aux_train_data=aux_train_data, aux_valid_data=aux_valid_data, L1=l1, L2=l2,
                         batch_size=batch_size, learning_rates=learning_rates, learning_rate_schedule=lrn_schedule,
                         optimiser=optimiser, max_pools=max_pools, lin_response_norms=lrns,
                         lin_response_alphas=lrn_alphas, batch_norms=batch_norms, bifurcation_point=bifurcation_point,
                         optimiser_params=optimizer_params, activation=activation, dropout=dropout,
                         epochs=epochs, error=error, input_image_size=image_size, name=model_name, joint_loss=joint_loss,
                                       soft_loss=soft_loss)
    
    model.create_network()
    model.initialize_network(logs_dir)
    model.train_model()
    
    ########### MTL CLASSIFIER CIFAR100 & CIFAR20 #################
    tf.reset_default_graph()
    model_name = "MTL_CIFAR_100_and_20_ADAM"
    print("MODEL:- " + model_name)
    
    main_train_data = CIFAR100DataProvider(which_set='train',
                                                    batch_size=batch_size, shuffle_order=False)
    main_valid_data = CIFAR100DataProvider(which_set='valid', batch_size=batch_size, shuffle_order=False)

    aux_train_data = CIFAR100DataProvider(which_set='train', use_coarse_targets=True,
                                                   batch_size=batch_size, shuffle_order=False)
    aux_valid_data = CIFAR100DataProvider(which_set='valid', use_coarse_targets=True,
                                          batch_size=batch_size, shuffle_order=False)
    
    m_t_idx = utils.sample_data(main_train_data, ratio)
    m_v_idx = utils.sample_data(main_valid_data, ratio)
    _1 = utils.sample_data(aux_train_data, ratio, m_t_idx)
    _2 = utils.sample_data(aux_valid_data, ratio, m_v_idx)
    
    model = TwoTaskConvModelSoftSharing(conv_layer_dims=layer_dims, main_train_data=main_train_data, main_valid_data=main_valid_data,
                         aux_train_data=aux_train_data, aux_valid_data=aux_valid_data, L1=l1, L2=l2,
                         batch_size=batch_size, learning_rates=learning_rates, learning_rate_schedule=lrn_schedule,
                         optimiser=optimiser, max_pools=max_pools, lin_response_norms=lrns,
                         lin_response_alphas=lrn_alphas, batch_norms=batch_norms, bifurcation_point=bifurcation_point,
                         optimiser_params=optimizer_params, activation=activation, dropout=dropout,
                         epochs=epochs, error=error, input_image_size=image_size, name=model_name, joint_loss=joint_loss,
                                       soft_loss=soft_loss)
    
    model.create_network()
    model.initialize_network(logs_dir)
    model.train_model()
    
    ########### CIFAR100 BASELINE #####################
    tf.reset_default_graph()
    model_name = 'baseline_with_' + str(ratio) + "_of_the_data_ADAM"
    print("MODEL:- " + model_name)
    
    train_data = CIFAR100DataProvider(which_set='train', batch_size=batch_size)
    valid_data = CIFAR100DataProvider(which_set='valid', batch_size=batch_size)
    
    # update baseline data providers to have same samples 
    _3 = utils.sample_data(train_data, ratio, m_t_idx)
    _4 = utils.sample_data(valid_data, ratio, m_v_idx)
    
    model_b = ConvModel(conv_layer_dims=layer_dims, train_data=train_data, valid_data=valid_data,
                 batch_size=batch_size, learning_rates=learning_rates, learning_rate_schedule=lrn_schedule,
                 optimiser=optimiser, L1=l1, L2=l2,
                 max_pools=max_pools, lin_response_norms=lrns, lin_response_alphas=lrn_alphas, batch_norms=batch_norms,
                 optimiser_params=optimizer_params, activation=activation, dropout=dropout,
                 epochs=epochs, error=error, input_image_size=image_size, name=model_name)
    model_b.create_network()
    model_b.initialize_network(logs_dir)
    model_b.train_model()

MODEL:- MTL_CIFAR_100_and_10_ADAM
Setting up model...


NotImplementedError: 